text dataset - LOTRs, IMDb movie plot summaries 

Word2vec
Doc2vec 

# Word2Vec

In [3]:
import os
import nlp_utils
import pandas as pd

/Applications/anaconda3/lib/python3.7/site-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping


In [40]:
path = '/Users/Paresh.Sharma@ibm.com/Paresh/Data_Science/Datasets/LOTRs_books'
os.chdir(path)

In [41]:
new_list = []
for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith('.txt'):
            with open(os.path.join(root, file), 'r', encoding='ISO-8859-1') as f:
                text = f.read()
                new_list.append(text)

In [42]:
text = ''.join(new_list)

In [44]:
# pre-process text
processed_tokens = nlp_utils.preprocess(text, n_grams=(1,2), remove_accents=True, lower=True, remove_less_than=1, remove_more_than=15,
                                  remove_punct=True, remove_alpha=True, remove_stopwords=True, 
                                  add_custom_stopwords = ['none'], lemma=False, stem=False, remove_url=False)

In [46]:
processed_tokens[:10]

['three',
 'rings',
 'elvenkings',
 'sky',
 'seven',
 'dwarflords',
 'halls',
 'stone',
 'nine',
 'mortal']

In [47]:
from gensim.models import Word2Vec

# train model 
model = Word2Vec(processed_tokens, size=1000, window=5, min_count=2, workers=6, sg=1, iter=10)

In [50]:
model.wv.most_similar('rivendell', topn = 20)

[('lorien', 0.933873176574707),
 ('moria', 0.9229995012283325),
 ('lothlorien', 0.9189971089363098),
 ('mordor', 0.9182309508323669),
 ('dain', 0.9102993011474609),
 ('isengard', 0.8989975452423096),
 ('bree', 0.8874597549438477),
 ('orthanc', 0.8804347515106201),
 ('nimrodel', 0.8768280148506165),
 ('fangorn', 0.8731997013092041),
 ('listens', 0.8729000091552734),
 ('mundburg', 0.8728089332580566),
 ('fret', 0.866926908493042),
 ('tarries', 0.8637433052062988),
 ('booming', 0.8634979128837585),
 ('rohan', 0.8630837202072144),
 ('dorthonion', 0.861647367477417),
 ('mastery', 0.8610222339630127),
 ('archery', 0.8580681681632996),
 ('shelob', 0.8580402731895447)]

In [49]:
model.wv.similarity('mordor', 'sauron')

0.86853987

# Doc2Vec

In [14]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

In [15]:
path = '/Users/Paresh.Sharma@ibm.com/Paresh/Data_Science/Datasets/Movie plots/'
os.chdir(path)

In [16]:
df = pd.read_csv('movie_plots.csv')

In [17]:
# only keep training data
df = df[df.split != 'val']

# delete split column
del df['split']

# reset the index 
df = df.reset_index(drop=True)

print(df.shape)
df.head()

(12455, 5)


,imdb_id,title,plot_synopsis,tags,synopsis_source
0,tt0057603,I tre volti della paura,Note: this synopsis is for the orginal Italian...,"cult, horror, gothic, murder, atmospheric",imdb
1,tt1733125,Dungeons & Dragons: The Book of Vile Darkness,"Two thousand years ago, Nhagruul the Foul, a s...",violence,imdb
2,tt0033045,The Shop Around the Corner,"Matuschek's, a gift store in Budapest, is the ...",romantic,imdb
3,tt0113862,Mr. Holland's Opus,"Glenn Holland, not a morning person by anyone'...","inspiring, romantic, stupid, feel-good",imdb
4,tt0249380,Baise-moi,Baise-moi tells the story of Nadine and Manu w...,"gothic, cruelty, violence, cult, revenge, sadist",wikipedia


In [18]:
processed_docs = df['plot_synopsis'].apply(nlp_utils.preprocess, n_grams=(1,1), remove_accents=True, lower=True, remove_less_than=1, remove_more_than=15,
                                  remove_punct=True, remove_alpha=True, remove_stopwords=True, 
                                  add_custom_stopwords = ['none'], lemma=False, stem=False, remove_url=True)

processed_docs = processed_docs.tolist()

In [19]:
tagged_data = [TaggedDocument(words=tokenised_plot, tags=[str(i)]) for i,tokenised_plot  in enumerate(processed_docs)]

In [20]:
max_epochs = 10

model = Doc2Vec(size=20,
                alpha=0.025, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    
    # decrease the learning rate
    #model.alpha -= 0.002
    # fix the learning rate, no decay
    #model.min_alpha = model.alpha

/Applications/anaconda3/lib/python3.7/site-packages/gensim/models/doc2vec.py:319: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


iteration 0


/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  from ipykernel import kernelapp as app


iteration 1


/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  from ipykernel import kernelapp as app


...

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  from ipykernel import kernelapp as app


iteration 8


/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  from ipykernel import kernelapp as app


iteration 9


/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  from ipykernel import kernelapp as app


In [21]:
model.save("/Users/Paresh.Sharma@ibm.com/Paresh/Data_Science/Github/DS-Projects/11_NLP_basics_practice/d2v.model")
print("Model Saved")

Model Saved


In [22]:
from gensim.models.doc2vec import Doc2Vec

model= Doc2Vec.load("/Users/Paresh.Sharma@ibm.com/Paresh/Data_Science/Github/DS-Projects/11_NLP_basics_practice/d2v.model")

In [23]:
# get a movie which is named in the df and find similar movies based on the description

def get_similar_movies_from_data(movie_name, df, topn):

    index_array = df.loc[df['title'] == movie_name].index.values

    if len(index_array) == 0:
        print('This movie is not in the database')

    else:
        idx = index_array[0]

        similar_doc = model.docvecs.most_similar(idx, topn = topn)
        indexes = [tup[0] for tup in similar_doc]
        return df.loc[df.index.isin(indexes)]['title'].values

In [24]:
get_similar_movies_from_data("Scarface", df, topn=20)

array(['Bugsy', 'The Sting', 'Bruiser', 'The Drop', 'Public Enemies',
       'Notorious', 'Rio Bravo', 'Falling Down', 'Pride and Glory',
       'American Hustle', 'Milk', 'The Great Flamarion', 'Four Brothers',
       "C'era una volta il West", 'Lucky Number Slevin', 'Payback',
       'Bugs and Thugs', 'Sherlock Jr.', 'Racketeer Rabbit',
       'Continuavano a chiamarlo Trinità'], dtype=object)

In [25]:
the_dark_knight = "Gotham's new District Attorney has been elected. His name is Harvey Dent, and he has a \
radical new agenda that threatens to take down Gotham's organized crime underworld once and for all with \
an iron fist. But the emergence of the rogue vigilante known as Batman has caused problems for Dent and his \
agenda. A new criminal mastermind known only as 'The Joker' has arrived and aims to take Gotham out from under \
Harvey Dent's iron fist. The Joker stages a masterfully planned bank robbery and robs the Gotham mob blind. He \
uses this money to stage a series of horrific and strategic attacks against the city and its people, each one \
carefully planned and aimed at Dent and Batman, while causing the rest of the city to enter panic mode. Meanwhile, \
Batman thinks he might have found a lead to The Joker thanks to Wayne Enterprises' dealings with a shady Chinese \
banker, and that takes Batman and Alfred to Hong Kong. The Joker has no rules, but Batman has only one, and the \
Joker aims to make Batman break his only rule. But who will be the one to take him out, will it be rogue vigilante \
Batman, or will it be elected official Harvey Dent, the new hero with a face?"

In [43]:
def get_similar_movies_from_new(plot_synoposis, df, topn):
    
    # pre-process text
    processed_tokens = nlp_utils.preprocess(plot_synoposis, n_grams=(1,1), remove_accents=True, lower=True, remove_less_than=1, 
                                            remove_more_than=15,remove_punct=True, remove_alpha=True, 
                                            remove_stopwords=True, add_custom_stopwords = ['none'], 
                                            lemma=False, stem=False, remove_url=True)
    
    new_vector = model.infer_vector(processed_tokens)
    similar_doc = model.docvecs.most_similar([new_vector], topn=topn)
    indexes = [tup[0] for tup in similar_doc]
    return list(df.loc[df.index.isin(indexes)]['title'].values)

In [44]:
get_similar_movies_from_new(the_dark_knight, df, topn=20)

['Batman Begins',
 'Batman',
 'Max Payne',
 'Batman: Arkham City',
 'Batman: The Dark Knight Returns, Part 2',
 'Batman: Under the Red Hood',
 'Spider-Man',
 'Batman: Mask of the Phantasm',
 'Batman Forever',
 'The Dark Knight',
 'The Dark Knight Rises',
 'Batman: The Killing Joke',
 'Justice League: Doom',
 'Batman: Arkham Origins',
 'Batman Begins',
 'Lego Batman: The Movie - DC Super Heroes Unite',
 'Tetsuo: The Bullet Man',
 'Batman: Return of the Caped Crusaders',
 'Son of Batman',
 'Batman: Year One']

## TF-IDF

In [96]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
df.head()

,imdb_id,title,plot_synopsis,tags,synopsis_source
0,tt0057603,I tre volti della paura,Note: this synopsis is for the orginal Italian...,"cult, horror, gothic, murder, atmospheric",imdb
1,tt1733125,Dungeons & Dragons: The Book of Vile Darkness,"Two thousand years ago, Nhagruul the Foul, a s...",violence,imdb
2,tt0033045,The Shop Around the Corner,"Matuschek's, a gift store in Budapest, is the ...",romantic,imdb
3,tt0113862,Mr. Holland's Opus,"Glenn Holland, not a morning person by anyone'...","inspiring, romantic, stupid, feel-good",imdb
4,tt0249380,Baise-moi,Baise-moi tells the story of Nadine and Manu w...,"gothic, cruelty, violence, cult, revenge, sadist",wikipedia


In [66]:
processed_docs = df['plot_synopsis'].apply(nlp_utils.preprocess, n_grams=(1,1), remove_accents=False, lower=True, remove_less_than=0, remove_more_than=15,
                                  remove_punct=True, remove_alpha=True, remove_stopwords=False, 
                                  add_custom_stopwords = [], lemma=False, stem=False, remove_url=False)

processed_docs = processed_docs.tolist()

In [67]:
the_dark_knight = "Gotham's new District Attorney has been elected. His name is Harvey Dent, and he has a \
radical new agenda that threatens to take down Gotham's organized crime underworld once and for all with \
an iron fist. But the emergence of the rogue vigilante known as Batman has caused problems for Dent and his \
agenda. A new criminal mastermind known only as 'The Joker' has arrived and aims to take Gotham out from under \
Harvey Dent's iron fist. The Joker stages a masterfully planned bank robbery and robs the Gotham mob blind. He \
uses this money to stage a series of horrific and strategic attacks against the city and its people, each one \
carefully planned and aimed at Dent and Batman, while causing the rest of the city to enter panic mode. Meanwhile, \
Batman thinks he might have found a lead to The Joker thanks to Wayne Enterprises' dealings with a shady Chinese \
banker, and that takes Batman and Alfred to Hong Kong. The Joker has no rules, but Batman has only one, and the \
Joker aims to make Batman break his only rule. But who will be the one to take him out, will it be rogue vigilante \
Batman, or will it be elected official Harvey Dent, the new hero with a face?"

In [159]:
# pre-process text
processed_tokens = nlp_utils.preprocess(the_dark_knight, n_grams=(1,1), remove_accents=False, lower=True, remove_less_than=0, remove_more_than=15,
                                  remove_punct=True, remove_alpha=True, remove_stopwords=False, 
                                  add_custom_stopwords = [], lemma=False, stem=False, remove_url=False)

In [160]:
processed_tokens = [processed_tokens]

In [162]:
def identity_tokenizer(text):
    return text

tfidf = TfidfVectorizer(tokenizer=identity_tokenizer, lowercase=False)    
tfidf_weights_matrix = tfidf.fit_transform(processed_docs)
search_query_weights = tfidf.transform(processed_tokens)

In [167]:
cosine_distance = cosine_similarity(search_query_weights, tfidf_weights_matrix)
similarity_list = list(cosine_distance[0])

In [173]:
# get the indexes of the most similar 
indexes = sorted(range(len(similarity_list)), key=lambda i: similarity_list[i])[-20:]

In [175]:
df.loc[df.index.isin(indexes)]['title'].values

array(['Batman Beyond: Return of the Joker', 'Batman Begins',
       'The Batman vs. Dracula', 'Batman', 'Batman: Arkham City',
       'Batman: The Dark Knight Returns, Part 2',
       'Batman: Under the Red Hood', 'Batman: Mask of the Phantasm',
       'Batman Forever', 'The Dark Knight', 'Batman: The Killing Joke',
       'Full Metal Jacket', 'Justice League: Doom',
       'Batman: Arkham Origins', 'Batman Begins',
       'Lego Batman: The Movie - DC Super Heroes Unite',
       'Batman: Return of the Caped Crusaders', 'Batman: Arkham Knight',
       'Batman: Arkham Asylum', 'Batman: Year One'], dtype=object)

Sources: 
https://radimrehurek.com/gensim/models/word2vec.html
https://rare-technologies.com/word2vec-tutorial/#app
https://medium.com/@mishra.thedeepak/doc2vec-simple-implementation-example-df2afbbfbad5
https://www.imdb.com/title/tt0468569/plotsummary?ref_=tt_stry_pl